### Katie Bisson | Code Sample ####

As my work is strictly confidential, I am not able to share a recent code sample from projects done at work.

You can find below an extension of a course assignment from my Data Structures and Algorithms course, in which I construct and test a kDTree class.

The kDTree is a balanced tree, and the class is designed to include user methods for a circular range query and a kNN query. kDTrees store data by spatial proximity across k dimensions, making them very useful for spatial data.

The data input in mind is a list of tuples - (x, y) coordinates, making k=2. Elements of the class are written to be adaptable for a higher dimension.

In [4]:
import math

class kDTree():
    
    # define internal variables
    _found_circular = [] # will append circular range query results
    _found_nearest_neighbors = [] # will append kNN query results
    _INF = float('inf')
    _maxdist = _INF # global maximum distance variable initialized to infinity, used in kNN query
    
    # initialize points, dimension (k)
    def __init__(self, points, dimension=2):
        self.points = points
        self.dimension = dimension
    
    # create the internal node class to use within trees
    # node is created for a given point with a left and right child
    class kDTreeNode():
        
        # initialize class and features
        # each node contains the point, the left child, and the right child
        def __init__(self, point, left, right):
            self.x = point[0] # to return x coordinate of node
            self.y = point[1] # to return y coordinate of node
            self.point = point
            self.left = left
            self.right = right
        
        # the printable representation of a node is a string of the coordinates
        def __repr__(self):
            return str(str(self.x) + "," + str(self.y))
    
    # function to create and sort tree, splitting on median
    # depth initialized to 0, and will increment recursively
    def _sort_tree(self, points, depth=0):
        
        # termination case for recursion
        if len(points) == 0:
            return
    
        axis = depth % self.dimension # keeping track of axis - in 2D x=0, y=1
        points.sort(key=lambda p: p[axis]) # sort by the current axis
        mid = len(points)//2 # the midpoint/median
        
        # recursively fill out tree
        l = self._sort_tree(points[:mid], depth+1)
        r = self._sort_tree(points[mid+1:], depth+1)
       
        return self.kDTreeNode(point=points[mid], left=l, right=r)
    
    # user method to return sorted tree
    def return_sorted_tree(self):
        
        tree = self._sort_tree(self.points)
        return tree # returns the root node
    
    ### Circular Range Query ###
    
    # helper function - search for all points in given radius, return points found
    def _range_query_circular(self, node, p, r, found, depth=0):
 
        # p = the point around which to search
        # node = the node in question, starting with the kD tree root node
    
        # pull out the x and y coordinates
        px = p[0]
        py = p[1]

        # termination case for recursion
        if node is None:
            return self._found_circular
        
        # find the radius distance from x, y points
        rminus = [px-r, py-r]
        rplus = [px+r, py+r]
        
        # recursively search either the left or right branch of the node
        # (whichever is in bounds)
        
        # reference the proper depth (e.g. x or y) 
        if p[depth % self.dimension] < rminus[depth % self.dimension]:
            self._range_query_circular(node.right, p, r, found, depth+1)
            return found
        if p[depth % self.dimension] > rplus[depth % self.dimension]:
            self._range_query_circular(node.left, p, r, found, depth+1)
            return found
        
        # distance between node point and current point
        distance = math.sqrt((px-node.x)**2 + (py - node.y)**2)
        
        # if point is within radius, it is within range - add to "found"
        if distance <= r:
            found.append((node.x, node.y))
        
        # search recursively on left and right branches/traverse tree
        self._range_query_circular(node.left, p, r, found, depth+1)
        self._range_query_circular(node.right, p, r, found, depth+1)
        
        return found
    
    # user method to return the points within a given radius
    def return_range_query_circular(self, p, r=5, depth=0):
        
        node = self.return_sorted_tree()
        self._found_circular = [] # if the query is run again, sets list to empty
        rqc = self._range_query_circular(node, p, r=5, found=self._found_circular)
        return rqc

    ### kNN Query ###
    
    # helper function - updates the list of found neighbors; updates and returns the maxdist
    # before you get to n elements, maxdist will be INF
    # when you get to n elements, maxdist will be the largest distance/last element of neighbors list
    def _update_neighbors(self, p0, p, neighbors, n):
        
        # p0 = the (x, y) point associated with a node that is being considered as a neighbor
        # p = point around which to search
        
        # pull out the x and y coordinates as variables
        px = p[0]
        py = p[1]
        p0x = p0[0]
        p0y = p0[1]
        d = math.sqrt((px-p0x)**2 + (py-p0y)**2) # distance variable between p and p0

        # i = index, x = list element in neighbors
        # neighbors are entered as a point, distance pair into the list
        for i, x in enumerate(neighbors):
            
            # if last element is reached, function returns distance of last element to p
            if i == n:
                return neighbors[-1][1]
            
            # if the distance is less than the distance in neighbors, add to list
            if d < x[1]:
                # inserted smaller distance before larger - will always be sorted by distance
                neighbors.insert(i, [p0, d])
                
                # after insertion, for loop is broken 1 of 2 ways:
                # 1: maxdist = inf if neighbor list hasn't reached n
                if len(neighbors) < n:
                    return self._INF
                # 2: otherwise update maxdist with last element in neighbors
                return neighbors[-1][1]

        #when list is empty, add first point to list
        neighbors.append([p0, d])
        return self._INF
    
    # helper function that performs the nearest neighbor search
    # for each node, want to check if it should be added to the neighbor list     
    def _nnquery(self, node, p, n, found, depth=0):
        
        # p = point around which to search
        # node = current tree root node, starting with the full kD tree   
        
        # stop searching further if node is empty
        if node is None:
            return
        
        # for leaf node, check if node point should be added into neighbors
        if node.left == None and node.right == None:
            self._maxdist = self._update_neighbors(node.point, p, found, n)
            return
        
        # get x vs y axis
        axis = depth % self.dimension
        
        # the nearer subtree is determined based on axis in question
        if p[axis] < node.point[axis]:
            nearer_tree, farther_tree = node.left, node.right
        else:
            nearer_tree, farther_tree = node.right, node.left
        
        self._nnquery(nearer_tree, p, n, found, depth+1) # recursively query nearer subtree
        self._maxdist = self._update_neighbors(node.point, p, found, n) # update maxdist
        
        # for farther subtree, check if distance is smaller than current maxdist
        # if so, then we'll search this subtree as well
        if abs(node.point[axis] - p[axis]) < self._maxdist: # must check the far side
            self._nnquery(farther_tree, p, n, found, depth+1)

    # user method that returns the n closest neighbors to a given point and their distance
    def return_nearest_neighbor(self, p, n=1):
        
        node = self.return_sorted_tree()
        self._found_nearest_neighbors = [] # ensure former searches are removed
        self._nnquery(node, p, n, self._found_nearest_neighbors)
        return self._found_nearest_neighbors[:n] #[0][0] - add if you want to return just the point for n=1

In [6]:
# TEST DATA:
points = [(2,2), (0,5), (9,13), (7,2), (15,5), (1,1)]
tree = kDTree(points)
tree.return_sorted_tree()

7,2

In [7]:
# notice that the (7,2) point - on the edge of the radius - is included in the query response
# this is a good indicator of proper function design
tree.return_range_query_circular((2,2))

[(7, 2), (2, 2), (1, 1), (0, 5)]

In [8]:
tree.return_nearest_neighbor((0,1))

[[(1, 1), 1.0]]